In [ ]:
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import ensemble
from sklearn import tree
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = pd.DataFrame(pd.read_csv("40k_modified.csv"))

In [ ]:
df.columns = ["ind","text","label"]

In [ ]:
df = df.drop(columns=["ind"])

In [ ]:
corpus = []
for i in df["text"]:
    corpus.append(i)
corpus = corpus[0:5000]

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
filter = set(stopwords.words("english"))

In [ ]:
final_corpus = []
for i in corpus:
    app_str = ""
    for x in i.split(" "):
      if x.lower() not in filter:
        app_str = app_str + x + " "
    final_corpus.append(app_str.lower())
final_corpus[0]


'legal husband rape wife '

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer="word",ngram_range=(1,3))
X = vectorizer.fit_transform(corpus)

In [ ]:
xDense = X.todense()
xDenseList=xDense.tolist()

In [ ]:
features = vectorizer.get_feature_names()
df2 = pd.DataFrame(xDenseList,columns=features)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
df = df[0:5000]
df2["Label"] = list(df.label)

In [ ]:
x_train = df2.sample(frac=0.9,random_state=0)

In [ ]:
x_test = df2.drop(x_train.index)

In [ ]:
df_hate = df2[df2.Label==1]
df_nothate = df2[df2.Label==0]

In [ ]:
x_train = df_hate.sample(frac=0.9,random_state=0)
y_train = df_nothate.sample(frac=0.9,random_state=0)

In [ ]:
x_test = df_hate.drop(x_train.index)
y_test = df_nothate.drop(y_train.index)

In [ ]:
x_train_df = pd.concat([x_train,y_train],axis=0)

In [ ]:
x_train_text = x_train_df.drop(["Label"],axis=1)

In [ ]:
y_train_label = list(x_train_df.Label)

In [ ]:
x_test_df = pd.concat([x_test,y_test],axis=0)

In [ ]:
x_test_text = x_test_df.drop(["Label"],axis=1)
y_test_label = x_test_df.Label

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(x_train_text,y_train_label)

LogisticRegression()

In [ ]:
predict=logreg.predict(x_test_text)
from sklearn.metrics import accuracy_score
accuracy_score(y_test_label,predict)


0.938

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb = GaussianNB()

In [ ]:
gnb.fit(x_train_text,y_train_label)

GaussianNB()

In [ ]:
predictGNB = gnb.predict(x_test_text)
accuracy_score(y_test_label,predictGNB)

0.946

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(max_depth=2,random_state=0)

In [ ]:
rfc.fit(x_train_text,y_train_label)

RandomForestClassifier(max_depth=2, random_state=0)

In [ ]:
predictRFC = rfc.predict(x_test_text)
accuracy_score(y_test_label,predictRFC)

0.666

In [ ]:
import pickle
with open("model_gnb","wb") as f:
    pickle.dump(gnb,f)

In [ ]:
import pickle
with open("model_rfc","wb") as f:
    pickle.dump(rfc,f)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
mnb = MultinomialNB()
mnb.fit(x_train_text,y_train_label)
predictMNB = mnb.predict(x_test_text)
accuracy_score(y_test_label,predictMNB)

0.944

In [ ]:
import pickle
with open("model_mnb","wb") as f:
    pickle.dump(rfc,f)

In [ ]:
import pickle
with open("model_lr","wb") as f:
    pickle.dump(logreg,f)

In [ ]:
print(accuracy_score(y_test_label,predictMNB))
print(accuracy_score(y_test_label,predictRFC))
print(accuracy_score(y_test_label,predictGNB))
print(accuracy_score(y_test_label,predict)) #logisticRegression

0.944
0.666
0.946
0.938


In [ ]:
from sklearn.ensemble import *

In [ ]:
estimators = [("Multinomial Naive Bayes",mnb),("Gaussian Naive Bayes",gnb),("Logistic Regression",logreg)]
ensemble = VotingClassifier(estimators, voting="soft")
ensemble.fit(x_train_text,y_train_label)
predictENS = ensemble.predict(x_test_text)
accuracy_score(y_test_label,predictENS)

0.952

In [65]:
with open("model_ens","wb") as f:
    pickle.dump(ensemble,f)

In [ ]:
input1=["i hate my phone"]
vtz = CountVectorizer(analyzer="word",ngram_range=(1,3))
tvtz = vtz.fit_transform(input1)
newDf = pd.DataFrame(tvtz.todense().tolist(),columns=vtz.get_feature_names())
predDf = x_train_text[0:0]

li = []
for i in predDf.columns:
  if i in newDf.columns:
    li.append(newDf[i])
  else:
    li.append(0.0)

predDf.loc[0] = li
if(ensemble.predict(predDf)[0]):
  print("Hateful!")
else:
  print("No hate was detected")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer,

No hate was detected


In [64]:
xpickle = x_train_text[0:0]
with open("train","wb") as f:
  pickle.dump(xpickle,f)